Add training py scripts to path

In [1]:
import sys
import os

path_prefix = "/home/kanthoulis/spider/"

training_scripts_dir = path_prefix + "training"
transforms_dir = path_prefix + "transforms"
image_dir = path_prefix + "image"

sys.path.append(training_scripts_dir)
sys.path.append(transforms_dir)
sys.path.append(image_dir)


Dependencies

In [2]:
#UNet 
import torch
from torch import nn
from torch.utils.data import DataLoader
import json 
import numpy as np
import re 
import os
import pathlib
from natsort import natsorted
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

#model imports 
import unet

import dataset_torch_numpy as dataset
import dataset_torch_numpy_aug as dataset_aug
import metric
#import epoch as ep #ep not to conlfict with var name in loop

#Set GPU/Cuda Device to run model on
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")
torch.manual_seed(46)

Using cuda device


/home/kanthoulis/jupyter-venv/lib/python3.12/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.24 (you have 1.4.23). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Load tensor data 

In [3]:
json_path = "/home/kanthoulis/spider/tensor_data/tensor_data.json" 

#Load tensor parameters from .json
with open(json_path, 'r') as file:
    data = json.load(file)

#Assign each value to a variable
row_max = data["row_max"]
col_max = data["col_max"]
image_tensor_min = data["image_tensor_min"]
image_tensor_max = data["image_tensor_max"]
label_tensor_min = data["label_tensor_min"]
label_tensor_max = data["label_tensor_max"]
masks_no = data["masks_no"]
masks_array = data["masks_array"]

print(masks_no)

20


Data Load

In [4]:
path_prefix = "/home/kanthoulis/spider/dataset/"

train_img_slice_dir = pathlib.Path(path_prefix + "train_image_numpy") #TODO SET PATH IN JHUB
train_label_slice_dir = pathlib.Path(path_prefix + "train_label_numpy") #TODO SET PATH IN JHUB 
test_img_slice_dir = pathlib.Path(path_prefix + "test_image_numpy") #TODO SET PATH IN JHUB
test_label_slice_dir= pathlib.Path(path_prefix + "test_label_numpy") 

#Create lists of filenames in directories (str)
image_path = train_img_slice_dir
label_path = test_label_slice_dir

image_dir_list = os.listdir(image_path)
label_dir_list = os.listdir(label_path)

#Sort lists
image_dir_list = natsorted(image_dir_list)
label_dir_list = natsorted(label_dir_list)

print("train dataset len",image_dir_list.__len__())
print("test dataset len",label_dir_list.__len__())

train dataset len 11484
test dataset len 3197


Dice Loss

In [5]:
def dice_loss(pred, target, smooth=1e-6):
    # Apply softmax to predictions (logits to probabilities)
    #print("dims", pred.shape)  
    pred = torch.softmax(pred, dim=1)  # (N, C, H, W)
    
    # Compute Dice Loss
    intersection = torch.sum(pred * target, dim=(2, 3))  # Sum over spatial dimensions
    union = torch.sum(pred, dim=(2, 3)) + torch.sum(target, dim=(2, 3))  # Union over spatial dims

    dice = (2.0 * intersection + smooth) / (union + smooth)
    return 1.0 - dice.mean()  # Return mean Dice loss


In [6]:
#CrossEntropyLoss
"""

class CombinedLoss(torch.nn.Module):
    def __init__(self, ce_weight=0.5, dice_weight=0.5):
        super(CombinedLoss, self).__init__()
        self.ce_weight = ce_weight
        self.dice_weight = dice_weight
        self.ce_loss = torch.nn.CrossEntropyLoss()
        #self.ce_loss = nn.BCEWithLogitsLoss() 

    def forward(self, outputs, targets):
        # Cross Entropy Loss expects class probabilities and the raw target labels (not one-hot)
        ce_loss = self.ce_loss(outputs, torch.argmax(targets, dim=1))
        
        # Dice Loss expects one-hot encoded targets and softmax probabilities
        dice_loss_val = dice_loss(outputs, targets)
        
        # Weighted sum of the two losses
        return self.ce_weight * ce_loss + self.dice_weight * dice_loss_val
"""

'\n\nclass CombinedLoss(torch.nn.Module):\n    def __init__(self, ce_weight=0.5, dice_weight=0.5):\n        super(CombinedLoss, self).__init__()\n        self.ce_weight = ce_weight\n        self.dice_weight = dice_weight\n        self.ce_loss = torch.nn.CrossEntropyLoss()\n        #self.ce_loss = nn.BCEWithLogitsLoss() \n\n    def forward(self, outputs, targets):\n        # Cross Entropy Loss expects class probabilities and the raw target labels (not one-hot)\n        ce_loss = self.ce_loss(outputs, torch.argmax(targets, dim=1))\n        \n        # Dice Loss expects one-hot encoded targets and softmax probabilities\n        dice_loss_val = dice_loss(outputs, targets)\n        \n        # Weighted sum of the two losses\n        return self.ce_weight * ce_loss + self.dice_weight * dice_loss_val\n'

In [7]:
#BCEWithLogits
class CombinedLoss(nn.Module):
    def __init__(self, ce_weight=0.5, dice_weight=0.5):
        super(CombinedLoss, self).__init__()
        self.ce_weight = ce_weight
        self.dice_weight = dice_weight
        self.ce_loss = nn.BCEWithLogitsLoss()

    def forward(self, outputs, targets):
        """
        Compute the combined loss.
        Args:
            outputs: Logits from the model (shape: B x C x H x W).
            targets: Ground truth labels (shape: B x C x H x W, binary).
        Returns:
            Combined loss value.
        """
        # Compute BCEWithLogitsLoss
        ce_loss = self.ce_loss(outputs, targets)
        
        # Compute Dice Loss
        dice_loss_val = dice_loss(outputs, targets)
        
        # Weighted sum of the two losses
        return self.ce_weight * ce_loss + self.dice_weight * dice_loss_val

Unet Torch - Model Parameters 

In [8]:
#Input Output channels
input_channels = 1 #Hounsfield scale, do not modify
output_channels = masks_no - 1 #-1 to exclude backround value 0, do not modify\

#Model Hyperparams 
depth = 5
start_filts = 32

up_mode = 'upsample'
lr = 0.0001 #adjust according to batch size 
batchsize = 16 #find optimal setting for jhub gpu(s)
#loss_func = nn.BCEWithLogitsLoss() 
#loss_func = nn.CrossEntropyLoss()
loss_func = CombinedLoss(ce_weight=0.5, dice_weight=0.5)
loss_func.to(device)

CombinedLoss(
  (ce_loss): BCEWithLogitsLoss()
)

Unet Torch - Model & Optimizer Init 

In [9]:
model = unet.UNet(in_channels= input_channels,num_classes=output_channels, depth= depth, start_filts=start_filts, up_mode=up_mode) 
#optim = torch.optim.Adam(model.parameters(), lr=lr)
model = nn.DataParallel(model) #parallelize between the 2 GPUs, not the best iteration but it works for this use case
model.to(device)
model.to(torch.float32)

optim = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.001) #AdamW


/home/kanthoulis/spider/training/unet.py:213: FutureWarning: `nn.init.xavier_normal` is now deprecated in favor of `nn.init.xavier_normal_`.
  init.xavier_normal(m.weight)
/home/kanthoulis/spider/training/unet.py:214: FutureWarning: `nn.init.constant` is now deprecated in favor of `nn.init.constant_`.
  init.constant(m.bias, 0)


Unet Torch - Load Model and Optim states for resuming training

In [10]:
#FIRST TRAINING SESSION ONLY
epoch_no = 0 

In [11]:
#NOTE: a simple script to get the file w the highest idx in the directory wouldn't be hard to iterate but for now
    #just focusing on training the model, maybe work on it further down the line 

#TODO also load scheduler when resuming training
"""
#last epoch trained path
path = "/home/kanthoulis/spider/models/spider_seg_29" 

checkpoint= torch.load(path)
print(checkpoint.keys())
#optim = torch.optim.Adam(model.parameters(), lr=lr)
optim = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.001)

def extract_number_from_path(path):
    match = re.search(r'(\d+)$', path)
    return int(match.group(1)) if match else None

epoch_no = extract_number_from_path(path) + 1 #number for plotting in tb
print(epoch_no)

model.load_state_dict(checkpoint['model_dict'])
optim.load_state_dict(checkpoint['optimizer_dict'])


model.to(device)
model.to(torch.float32)
"""

<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_606269/2501647748.py:5: SyntaxWarning: invalid escape sequence '\d'
  """


'\n#last epoch trained path\npath = "/home/kanthoulis/spider/models/spider_seg_29" \n\ncheckpoint= torch.load(path)\nprint(checkpoint.keys())\n#optim = torch.optim.Adam(model.parameters(), lr=lr)\noptim = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.001)\n\ndef extract_number_from_path(path):\n    match = re.search(r\'(\\d+)$\', path)\n    return int(match.group(1)) if match else None\n\nepoch_no = extract_number_from_path(path) + 1 #number for plotting in tb\nprint(epoch_no)\n\nmodel.load_state_dict(checkpoint[\'model_dict\'])\noptim.load_state_dict(checkpoint[\'optimizer_dict\'])\n\n\nmodel.to(device)\nmodel.to(torch.float32)\n'

UNet Torch - Datasets & Dataloaders

In [12]:
#Datasets 
#train_set = dataset.SpiderDatasetNumpy(train_label_slice_dir, train_img_slice_dir)
train_set = dataset_aug.SpiderDatasetNumpyAug(train_label_slice_dir, train_img_slice_dir)
test_set = dataset.SpiderDatasetNumpy(test_label_slice_dir, test_img_slice_dir)

#Dataloaders
train_dataloader = DataLoader(train_set, batch_size = batchsize, shuffle=True) 
test_dataloader = DataLoader(test_set, batch_size = batchsize, shuffle=True)

print(len(train_dataloader) * batchsize)
print(len(test_dataloader) * batchsize)

11488
3200


Accuracy Metrics

In [13]:
#BCE
metric_calculator = metric.SegmentationMetrics(average=True, ignore_background=True,activation='sigmoid') #for calculating 
metric_calculator_binary = metric.BinaryMetrics(activation='sigmoid') #for calculating spinal canal metrics since it's only 1 class
"""
#CrossEntropyLoss()
metric_calculator = metric.SegmentationMetrics(average=True, ignore_background=True,activation='softmax') #for calculating 
metric_calculator_binary = metric.BinaryMetrics() #for calculating spinal canal metrics since it's only 1 class
"""

"\n#CrossEntropyLoss()\nmetric_calculator = metric.SegmentationMetrics(average=True, ignore_background=True,activation='softmax') #for calculating \nmetric_calculator_binary = metric.BinaryMetrics() #for calculating spinal canal metrics since it's only 1 class\n"

LR Scheduler

In [14]:
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

# Scheduler setup
T_0 = len(train_dataloader)  # One epoch per cycle
T_mult = 2                   # Each subsequent cycle is 2x longer than the previous
eta_min = 1e-6               # Minimum learning rate

scheduler = CosineAnnealingWarmRestarts(
    optimizer=optim,
    T_0=T_0,
    T_mult=T_mult,
    eta_min=eta_min
)

#scheduler = CosineAnnealingLR(optim, T_max=len(train_dataloader) * epochs, eta_min=1e-6)

Batch Accumulation

In [15]:
accumulation_steps  = 4 #4*16 = 64

Set no. of epochs to train for training session

In [16]:
epochs = 50

Train Loop

In [ ]:
timestamp = datetime.now().strftime('%Y%m%d_%H')
#writer = SummaryWriter('runs/spider_seg_unet_epochs={}_lr={}_batchsize={}_loss=BCEWithLogits_startfilts={}_upmode={}'.format(epochs,lr, batchsize,start_filts,up_mode))
tb_writer = SummaryWriter('/home/kanthoulis/spider/runs/spider_seg'.format(timestamp)) #for creating new runs 
#tb_writer = SummaryWriter('/home/kanthoulis/spider/runs/spider_seg_20250106_13') #continue where left off
#tb_writer = SummaryWriter('/home/kanthoulis/spider/runs/junk') #continue where left off
epoch_number = epoch_no #set to no of last epoch completed to have consistent tb plots 

best_vloss = 1_000_000.

for epoch in range(epochs):
    print('EPOCH {}:'.format(epoch_number + 1))

    #----------------------Train Loop---------------------------------
    model.train(True) #set model to train
    #Instantiate training metrics per epoch
    running_loss = 0.
    last_loss = 0.

    running_accu = 0.
    running_dice = 0.
    #running_precision = 0.
    #running_recall = 0.

    vert_running_accu = 0.
    vert_running_dice = 0.

    sc_running_accu = 0.
    sc_running_dice = 0.

    ivd_running_accu = 0.
    ivd_running_dice = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting

    #swap train dataloader for dset
    for i, data in enumerate(tqdm(train_dataloader, desc="Training")):
        # Every data instance is an input + label pair
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero your gradients for every batch!
        optim.zero_grad()

        
        # Make predictions for this batch
        outputs = model(inputs)

        #print("outputs shape", outputs.shape)
        
        #input output label shape [8 batchsize, 19 classes, row, col]


        # Compute the loss and its gradients
        loss = loss_func(outputs, labels)
        """
        loss.backward()

        # Adjust learning weights
        optim.step()

        scheduler.step()
        """

                # Scale the loss by the accumulation steps
        loss = loss / accumulation_steps  # Scale down the loss by the number of accumulation steps

        # Backpropagate (accumulate gradients)
        loss.backward()

        # Only update weights after the specified number of accumulation steps
        if (i + 1) % accumulation_steps == 0:
            optim.step()  # Update model weights
            scheduler.step()  # Adjust the learning rate if necessary TESTING NO SCHEDULER 
            optim.zero_grad()  # Reset gradients for the next accumulation cycle
        
        #General metrics
        accu, dice, prec, recall = metric_calculator(labels, outputs)

        #Vertebrae Metrics (0-9)
        vert_labels = labels[:, :9, :, :]
        vert_outputs = outputs[:, :9, :, :]
        vert_accu, vert_dice, vert_prec, vert_recall = metric_calculator(vert_labels, vert_outputs)

        #Spinal Canal Metrics (10)
        sc_labels = labels[:, 10, :, :].unsqueeze(1)
        sc_outputs = outputs[:, 10, :, :].unsqueeze(1)
        sc_accu, sc_dice, sc_prec, sc_specificity, sc_recall = metric_calculator_binary(sc_labels, sc_outputs)

        #IVD Metrics (11-19)
        ivd_labels = labels[:, -9:, :, :]
        ivd_outputs = outputs[:, -9:, :, :]
        ivd_accu, ivd_dice, ivd_prec, ivd_recall = metric_calculator(ivd_labels, ivd_outputs)


        # Gather data and report
        #General
        running_loss += loss.item()
        running_accu += accu
        running_dice += dice
        #running_dice += dice
        #running_precision += prec
        #running_recall += recall

        #Vertebrae
        vert_running_accu += vert_accu
        vert_running_dice += vert_dice

        #Spinal Canal
        sc_running_accu += sc_accu
        sc_running_dice += sc_dice

        #IVD
        ivd_running_accu += ivd_accu
        ivd_running_dice += ivd_dice
    
        #if (i) % 50 == 0:
            #print("batch" ,i )

        # every 50 batches 
        if i % 50 == 49: 
            
            last_loss = running_loss / 50
            last_accu = running_accu / 50
            last_dice = running_dice / 50

            vert_last_accu = vert_running_accu / 50
            vert_last_dice = vert_running_dice / 50

            sc_last_accu = sc_running_accu / 50
            sc_last_dice = sc_running_dice / 50

            ivd_last_accu = ivd_running_accu / 50
            ivd_last_dice = ivd_running_dice / 50

            tb_x = epoch_number * len(train_dataloader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)

            tb_writer.add_scalar('General/accuracy_train', last_accu, tb_x)
            tb_writer.add_scalar('General/dice_train', last_dice, tb_x)

            tb_writer.add_scalar('Vertebrae/accuracy_train', vert_last_accu, tb_x)
            tb_writer.add_scalar('Vertebrae/dice_train', vert_last_dice, tb_x)

            tb_writer.add_scalar('Spinal Canal/accuracy_train', sc_last_accu, tb_x)
            tb_writer.add_scalar('Spinal Canal/dice_train', sc_last_dice, tb_x)

            tb_writer.add_scalar('Intervertebral Discs/accuracy_train', ivd_last_accu, tb_x)
            tb_writer.add_scalar('Intervertebral Discs/dice_train', ivd_last_dice, tb_x)

            running_loss = 0.
            running_accu = 0.
            running_dice = 0.

            vert_running_accu = 0.
            vert_running_dice = 0.

            sc_running_accu = 0.
            sc_running_dice = 0.

            ivd_running_accu = 0.
            ivd_running_dice = 0.

    #-----------------Eval loop------------------------------------
    #print("avg loss in epoch", avg_loss)

    # Initialize training metrics
    running_tloss = 0.0
    running_taccu = 0.0
    running_tdice = 0.0
    
    vert_running_taccu = 0.0
    vert_running_tdice = 0.0
    
    sc_running_taccu = 0.0
    sc_running_tdice = 0.0
    
    ivd_running_taccu = 0.0
    ivd_running_tdice = 0.0
    

    running_vloss = 0.0
    running_vaccu = 0.0
    running_vdice = 0.0

    vert_running_vaccu = 0.0
    vert_running_vdice = 0.0
    
    sc_running_vaccu = 0.0
    sc_running_vdice = 0.0

    ivd_running_vaccu = 0.0
    ivd_running_vdice = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():

        for t, tdata in enumerate(tqdm(train_dataloader, desc="Evaluating Training Set")):
            tinputs, tlabels = tdata
    
            # Make predictions for this batch
            toutputs = model(tinputs)
            
            # Compute loss
            tloss = loss_func(toutputs, tlabels)
            
            # General Metrics
            taccu, tdice, tprec, trecall = metric_calculator(tlabels, toutputs)
    
            # Vertebrae Metrics (0-9)
            vert_tlabels = tlabels[:, :9, :, :]
            vert_toutputs = toutputs[:, :9, :, :]
            vert_taccu, vert_tdice, vert_tprec, vert_trecall = metric_calculator(vert_tlabels, vert_toutputs)
    
            # Spinal Canal Metrics (10)
            sc_tlabels = tlabels[:, 10, :, :].unsqueeze(1)
            sc_toutputs = toutputs[:, 10, :, :].unsqueeze(1)
            sc_taccu, sc_tdice, sc_tprec, sc_tspecificity, sc_trecall = metric_calculator_binary(sc_tlabels, sc_toutputs)
    
            # IVD Metrics (11-19)
            ivd_tlabels = tlabels[:, -9:, :, :]
            ivd_toutputs = toutputs[:, -9:, :, :]
            ivd_taccu, ivd_tdice, ivd_tprec, ivd_trecall = metric_calculator(ivd_tlabels, ivd_toutputs)
    
            # Accumulate metrics
            running_tloss += tloss.item()
            running_taccu += taccu
            running_tdice += tdice
    
            vert_running_taccu += vert_taccu
            vert_running_tdice += vert_tdice
    
            sc_running_taccu += sc_taccu
            sc_running_tdice += sc_tdice
    
            ivd_running_taccu += ivd_taccu
            ivd_running_tdice += ivd_tdice


 
        #eval metrics
        for j, vdata in enumerate(tqdm(test_dataloader, desc="Evaluating Test Set")):

           # if( j % 20 == 0):
               # print("valuation test set batch", j)
            vinputs, vlabels = vdata

            voutputs = model(vinputs)
            vloss = loss_func(voutputs, vlabels)
            
            vaccu, vdice, vprec, vrecall = metric_calculator(vlabels, voutputs)

            #Vertebrae Metrics (0-9)
            vert_vlabels = vlabels[:, :9, :, :]
            vert_voutputs = voutputs[:, :9, :, :]
            vert_vaccu, vert_vdice, vert_vprec, vert_vrecall = metric_calculator(vert_vlabels, vert_voutputs)

            #Spinal Canal Metrics (10)
            sc_vlabels = vlabels[:, 10, :, :].unsqueeze(1)
            sc_voutputs = voutputs[:, 10, :, :].unsqueeze(1)
            sc_vaccu, sc_vdice, sc_vprec, sv_vspecif, sc_vrecall = metric_calculator_binary(sc_vlabels, sc_voutputs)

            #IVD Metrics (11-19)
            ivd_vlabels = vlabels[:, -9:, :, :]
            ivd_voutputs = voutputs[:, -9:, :, :]
            ivd_vaccu, ivd_vdice, ivd_vprec, ivd_vrecall = metric_calculator(ivd_vlabels, ivd_voutputs)


            running_vloss += vloss
            running_vaccu += vaccu
            running_vdice += vdice

            vert_running_vaccu += vert_vaccu
            vert_running_vdice += vert_vdice

            sc_running_vaccu += sc_vaccu
            sc_running_vdice += vert_vdice

            ivd_running_vaccu += ivd_vaccu
            ivd_running_vdice += ivd_vdice


    # Calculate averages
    avg_tloss = running_tloss / (t + 1)
    avg_taccu = running_taccu / (t + 1)
    avg_tdice = running_tdice / (t + 1)

    vert_avg_taccu = vert_running_taccu / (t + 1)
    vert_avg_tdice = vert_running_tdice / (t + 1)

    sc_avg_taccu = sc_running_taccu / (t + 1)
    sc_avg_tdice = sc_running_tdice / (t + 1)

    ivd_avg_taccu = ivd_running_taccu / (t + 1)
    ivd_avg_tdice = ivd_running_tdice / (t + 1)

    avg_vloss = running_vloss / (j + 1)
    avg_vaccu = running_vaccu / (j + 1)
    avg_vdice = running_vdice / (j + 1)

    vert_avg_vaccu = vert_running_vaccu / (j + 1)
    vert_avg_vdice = vert_running_vdice / (j + 1)

    sc_avg_vaccu = sc_running_vaccu / (j + 1)
    sc_avg_vdice = sc_running_vdice / (j + 1)

    ivd_avg_vaccu = ivd_running_vaccu / (j + 1)
    ivd_avg_vdice = ivd_running_vdice / (j + 1)


    print('Loss: Train {} Valid {}'.format(avg_tloss, avg_vloss))
    print('Accuracy: Train {} Valid {}'.format(avg_taccu, avg_vaccu))
    print('Dice: Train {} Valid {}'.format(avg_tdice, avg_vdice))
    # Log the running loss averaged per batch
    # for both training and validation
    
    tb_writer.add_scalars('Loss/train_vs_valid',
                    {'Training' : avg_tloss, 'Validation' : avg_vloss},
                    epoch_number + 1)
    
    tb_writer.add_scalars('General/accuracy_train_vs_valid',
                    {'Training' : avg_taccu, 'Validation': avg_vaccu},
                    epoch_number + 1)

    tb_writer.add_scalars('General/dice_train_vs_valid',
                    {'Training' : avg_tdice, 'Validation': avg_vdice},
                    epoch_number + 1)
    #vert
    tb_writer.add_scalars('Vertebrae/accuracy_train_vs_valid',
                    {'Training' : vert_avg_taccu , 'Validation': vert_avg_vaccu},
                    epoch_number + 1)

    tb_writer.add_scalars('Vertebrae/dice_train_vs_valid',
                    {'Training' : vert_avg_tdice, 'Validation': vert_avg_vdice},
                    epoch_number + 1)
    #spinal canal
    tb_writer.add_scalars('Spinal Canal/accuracy_train_vs_valid',
                    {'Training': sc_avg_taccu, 'Validation': sc_avg_vaccu},
                    epoch_number + 1)

    tb_writer.add_scalars('Spinal Canal/dice_train_vs_valid',
                    {'Training': sc_avg_tdice ,'Validation': sc_avg_vdice},
                    epoch_number + 1)
    #ivd
    tb_writer.add_scalars('Intervertebral Discs/accuracy_train_vs_valid',
                    {'Training' : ivd_avg_taccu ,'Validation': ivd_avg_vaccu},
                    epoch_number + 1)

    tb_writer.add_scalars('Intervertebral Discs/dice_valid',
                    {'Training' : ivd_avg_tdice ,'Validation': ivd_avg_vdice},
                    epoch_number + 1)
    
    tb_writer.flush()
    
    #Change path to save model accordingly     
    model_path = '/home/kanthoulis/spider/models/spider_seg_{}'.format(epoch_number)

    torch.save({
        'model_dict': model.state_dict(),
        'optimizer_dict': optim.state_dict(),
        'scheduler_dict': scheduler.state_dict()  
    }, model_path)
    
    #torch.save({'model_dict': model.state_dict(), 'optimizer_dict': optim.state_dict()}, model_path)
        
    epoch_number += 1

EPOCH 1:


Training:  21%|██        | 150/718 [01:28<05:31,  1.72it/s]

# SHUT DOWN KERNELS AFTER TRAINING TO FREE UP MEMORY